In [1]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.pool import NullPool
import matplotlib as plt
import matplotlib.pylab as plt
import seaborn as sns
from tqdm import tqdm
from PIL import Image
import requests
from io import BytesIO





In [20]:
q = '''
SELECT ref_term,
       count(*) AS imp_cnt
FROM bun_log_db.app_event_type_impression
WHERE device_type = 'a'
  AND YEAR||MONTH||DAY = '20201204'
  AND page_id = '검색결과'
  AND ref_source IN ('ad_super_up_shop',
                     'ad_super_up',
                     'power_up')
GROUP BY 1
'''
old_imp = pd.read_sql(q, con=bun_dw)

In [12]:
q = '''
SELECT ref_term,
       count(*) AS click_cnt
FROM bun_log_db.app_event_type_view
WHERE device_type = 'a'
  AND YEAR||MONTH||DAY = '20201204'
  AND ref_source IN ('ad_super_up_shop',
                     'ad_super_up',
                     'power_up')
GROUP BY 1
'''
old_click = pd.read_sql(q, con=bun_dw)

In [28]:
q = '''
SELECT search_term AS ref_term,
       count(*) AS search_cnt
FROM bun_log_db.app_event_type_search
WHERE device_type = 'a'
  AND YEAR||MONTH||DAY = '20201204'
GROUP BY 1;
'''
old_search = pd.read_sql(q, con=bun_dw)

In [9]:
q = '''
SELECT ref_term,
       count(*) AS imp_cnt
FROM bun_log_db.api_event_type_impression_ad
WHERE device_type = 'a'
  AND YEAR||MONTH||DAY = '20201218'
  AND page_id = '검색결과'
  AND ref_source IN ('sa',
                     'ca')
GROUP BY 1
'''
new_imp = pd.read_sql(q, con=bun_dw)

In [15]:
q = '''
SELECT ref_term,
       count(*) AS click_cnt
FROM bun_log_db.api_event_type_click_ad
WHERE device_type = 'a'
  AND YEAR||MONTH||DAY = '20201218'
  AND ref_source IN ('sa',
                     'ca')
GROUP BY 1
'''
new_click = pd.read_sql(q, con=bun_dw)

In [29]:
q = '''
SELECT search_term AS ref_term,
       count(*) AS search_cnt
FROM bun_log_db.api_event_type_search
WHERE device_type = 'a'
  AND YEAR||MONTH||DAY = '20201218'
GROUP BY 1
'''
new_search = pd.read_sql(q, con=bun_dw)

In [21]:
old = pd.merge(old_imp, old_click, on='ref_term', how='left')
new = pd.merge(new_imp, new_click, on='ref_term', how='left')

In [32]:
old = pd.merge(old, old_search, on='ref_term', how='left')
new = pd.merge(new, new_search, on='ref_term', how='left')

In [33]:
old.sort_values('imp_cnt', ascending=False).to_csv('./old_ad.csv')
new.sort_values('imp_cnt', ascending=False).to_csv('./new_ad.csv')

In [31]:
new_search

,ref_term,search_cnt
0,에르메스롱망토,1
1,닌텐도 위 게임,2
2,헬로키티 스트랩 고래,1
3,bmw538i 스피커,1
4,스쿠터쿠션,1
...,...,...
276412,대구 a90,4
276413,501연청,1
276414,sm7 보키,1
276415,뉴발327 블랙,2


In [35]:
q = '''
SELECT imp.ref_term,
       imp.imp_cnt,
       click.click_cnt,
       search.search_cnt
FROM
  (SELECT ref_term,
          count(*) AS imp_cnt
   FROM bun_log_db.app_event_type_impression
   WHERE device_type = 'a'
     AND YEAR||MONTH||DAY = '20201204'
     AND page_id = '검색결과'
     AND ref_source IN ('ad_super_up_shop',
                        'ad_super_up',
                        'power_up')
   GROUP BY 1) AS imp
LEFT JOIN
  (SELECT ref_term,
          count(*) AS click_cnt
   FROM bun_log_db.app_event_type_view
   WHERE device_type = 'a'
     AND YEAR||MONTH||DAY = '20201204'
     AND ref_source IN ('ad_super_up_shop',
                        'ad_super_up',
                        'power_up')
   GROUP BY 1) AS click ON imp.ref_term = click.ref_term
LEFT JOIN
  (SELECT search_term AS ref_term,
          count(*) AS search_cnt
   FROM bun_log_db.app_event_type_search
   WHERE device_type = 'a'
     AND YEAR||MONTH||DAY = '20201204'
   GROUP BY 1) AS SEARCH ON imp.ref_term = search.ref_term
'''
oldAd = pd.read_sql(q, con=bun_dw)

In [39]:
oldAd.sort_values('imp_cnt', ascending=False).to_csv('./old_ad.csv')

In [37]:
q = '''
SELECT imp.ref_term,
       imp.imp_cnt,
       click.click_cnt,
       search.search_cnt
FROM
  (SELECT ref_term,
          count(*) AS imp_cnt
   FROM bun_log_db.api_event_type_impression_ad
   WHERE device_type = 'a'
     AND YEAR||MONTH||DAY = '20201218'
     AND page_id = '검색결과'
     AND ref_source IN ('sa',
                        'ca')
   GROUP BY 1) AS imp
LEFT JOIN
  (SELECT ref_term,
          count(*) AS click_cnt
   FROM bun_log_db.api_event_type_click_ad
   WHERE device_type = 'a'
     AND YEAR||MONTH||DAY = '20201218'
     AND ref_source IN ('sa',
                        'ca')
   GROUP BY 1) AS click ON imp.ref_term = click.ref_term
LEFT JOIN
  (SELECT search_term AS ref_term,
          count(*) AS search_cnt
   FROM bun_log_db.api_event_type_search
   WHERE device_type = 'a'
     AND YEAR||MONTH||DAY = '20201218'
   GROUP BY 1) AS SEARCH ON imp.ref_term = search.ref_term
'''
newAd = pd.read_sql(q, con=bun_dw)

In [41]:
newAd.sort_values('imp_cnt', ascending=False).to_csv('./new_ad.csv')